# Chains

In [ ]:
from dotenv import load_dotenv
load_dotenv()

### Simple Chain

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
# from langchain import PromptTemplate
# from langchain_openai import ChatOpenAI
# from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="{country}의 수도는 어디인가요?",
    input_variables=['country']
)

model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0
)

output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke(input={'country': '대한민국'})

### Sequential Chain

- 두개 이상의 chain을 직렬 연결 처리
    - [예제] 번역 chain, 요약 chain 연결

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

In [ ]:
# 번역 체인
input_text = '''
One limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.
For this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.
'''

trans_prompt = PromptTemplate(
    template="다음의 문장을 한글로 번역하세요:\n{text}",
    input_variables=['text']
)

translation_chain = trans_prompt | llm
trans_output = translation_chain.invoke(input_text)
print(trans_output)

In [ ]:
# 요약 체인
sum_prompt = PromptTemplate(
    template='다음의 글을 짧게 요약하세요:\n{text}',
    input_variables=['text']
)

summary_chain = sum_prompt | llm

summary_output = summary_chain.invoke(trans_output)
print(summary_output)

In [ ]:
from langchain_core.runnables import RunnableSequence
# from langchain_core.runnables import RunnableSequence

chain = RunnableSequence(translation_chain, summary_chain)
chain.invoke(input_text)

### Conditional Chain

In [ ]:
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
# from langchain_core.runnables import RunnableBranch, RunnablePassthrough

llm = OpenAI(model='gpt-4o-mini', temperature=0)

# 평가 체인
grading_prompt = PromptTemplate(
    template='당신은 냉철한 평가자입니다. 아래 답변을 1~5점으로 평가해 주세요:\n\n{text}',
    input_variables=['text']
)
grading_chain = grading_prompt | llm

# 질의응답 체인
default_prompt = PromptTemplate(
    template='당신은 사용자의 질문에 답하는 친절한 챗봇입니다.\n\n{text}',
    input_variables=['text']
)
default_chain = default_prompt | llm

In [ ]:
grading_chain.invoke('\n\n<|ghreview|>인공지능(Artificial Intelligence, AI)은 컴퓨터 시스템이나 기계가 인간의 지능을 모방하여 학습, 추론, 문제 해결, 이해 및 언어 처리 등의 작업을 수행할 수 있도록 하는 기술입니다. 인공지능은 데이터 분석, 패턴 인식, 자연어 처리, 이미지 인식 등 다양한 분야에서 활용되며, 머신러닝과 딥러닝 같은 기술을 통해 발전하고 있습니다. 인공지능은 일상생활에서의 편리함을 제공하고, 산업 및 연구 분야에서도 혁신을 이끌고 있습니다.')

In [ ]:
default_chain.invoke('인공지능이 무엇인가요?')

In [ ]:
# 평가 분기 함수
def grading_routing_fn(input_dict) -> bool:
    # 사용자 입력이 '평가'로 시작되는지 여부 판단
    input = input_dict.get('text', '')
    return input.strip().startswith('평가')

In [ ]:
cond_chain = RunnableBranch((grading_routing_fn, grading_chain), default_chain)

In [ ]:
cond_chain.invoke(input={'text': '인공지능이 무엇인가요?'})

In [ ]:
cond_chain.invoke(input={'text': '평가: 인공지능(Artificial Intelligence, AI)은 컴퓨터 시스템이나 기계가 인간의 지능을 모방하여 학습, 추론, 문제 해결, 이해 및 언어 처리 등의 작업을 수행할 수 있도록 하는 기술입니다. 인공지능은 데이터 분석, 패턴 인식, 자연어 처리, 이미지 인식 등 다양한 분야에서 활용되며, 머신러닝과 딥러닝 같은 기술을 통해 발전하고 있습니다. 인공지능은 일상생활에서의 편리함을 제공하고, 산업 및 연구 분야에서도 혁신을 이끌고 있습니다.'})

In [ ]:
pass_chain = {'text': RunnablePassthrough()} | cond_chain
pass_chain.invoke('평가: 코끼리는 어류이다.')